<a href="https://colab.research.google.com/github/tyukei/LLMs-from-scratch/blob/feature%2Fch03/ch03/01_main-chapter-code/ch03_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href = "http://mng.bz/oryv"の補足コード>ゼロから大きな言語モデルを構築</a> <a href = "https://sebastianraschka.com"> sebastian raschka </a> <br><br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第3章：注意メカニズムのコーディング

このノートブックで使用されているパッケージ：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.6.0+cu124


- この章では、注意メカニズム、LLMSのエンジンをカバーしています。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1長いシーケンスのモデリングの問題

- このセクションにはコードがありません
- ソース言語とターゲット言語間の文法構造の違いにより、単語ごとにテキストワードを翻訳することは不可能ではありません。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- トランスモデルの導入前に、エンコーダーデコーダーRNNは一般的に機械翻訳タスクに使用されていました
- このセットアップでは、エンコーダはソース言語からのトークンのシーケンスを処理します。これは、入力シーケンス全体の凝縮された表現を生成するために、ニューラルネットワーク内の一種の中間層である隠された状態を使用します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2注意メカニズムを使用したデータ依存関係のキャプチャ

- このセクションにはコードがありません
- 注意メカニズムを通じて、ネットワークのテキスト生成デコーダーセグメントは、すべての入力トークンに選択的にアクセスでき、特定の入力トークンが特定の出力トークンの生成において他のものよりも重要であることを意味します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- 変圧器の自己関節は、同じシーケンス内の他のすべての位置の関連性に関与して決定するために、各位置をシーケンス内に有効にすることにより、入力表現を強化するように設計された技術です。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

##3.3自己告発で入力のさまざまな部分に参加する

### 3.3.1訓練可能なウェイトのない単純な自己触媒メカニズム

**シンプルアテンション機構の流れ**

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="200px">

**アテンションスコア[ω]: クエリとのドット積**

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="200px">


**アテンションウェイト[α]: アテンションスコアの正規化(足して１にする)**

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="200px">


**コンテキストベクトル[z]: アテンションウェイトとの加重合計**

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="200px">

- このセクションでは、非常に単純化された自己触媒のバリアントについて説明します。
- これは純粋にイラストの目的であり、トランスで使用される注意メカニズムではありません
- 次のセクション、セクション3.3.2は、この単純な注意メカニズムを拡張して、実際の自己触媒メカニズムを実装する
- 入力シーケンス$ x^{（1）} $ to $ x^{（t）} $が与えられているとします
- 入力はテキスト（たとえば、「あなたの旅は1つのステップで始まる」のような文です）は、第2章で説明されているように、すでにトークン埋め込みに変換されています
- たとえば、$ x^{（1）} $は、「your」などを表すD次元ベクトルです。
-  **目標：**コンテキストベクトル$ z^{（i）} $ for各入力シーケンス要素$ x^{（i）} $ in $ x^{（1）} $ to $ x^{（t）} $（$ z $と$ x $の次元が同じです）
- コンテキストベクトル$ Z^{（i）} $は、入力$ x^{（1）} $ $ x^{（t）} $に加えて加重合計です。
- コンテキストベクトルは、特定の入力に「コンテキスト」 - 特異的です
- 任意の入力トークンのプレースホルダーとして$ x^{（i）} $の代わりに、2番目の入力、$ x^{（2）} $を考えてみましょう
- そして、具体的な例を継続するために、プレースホルダー$ z^{（i）} $の代わりに、2番目の出力コンテキストベクトル、$ z^{（2）} $を検討します。
-2番目のコンテキストベクトル$ z^{（2）} $は、2番目の入力要素に対して加重されたすべての入力$ x^{（1）} $ to $ x^{（t）} $にわたって加重合計です。$ x^{（2）} $
- 注意の重みは、$ z^{（2）} $を計算する際に、各入力要素が加重合計にどれだけの寄与しているかを決定する重みです
- 要するに、$ z^{（2）} $は、手元の特定のタスクに関連する他のすべての入力要素に関する情報も組み込まれている$ x^{（2）} $の変更バージョンと考えてください

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- この図の数字は1つに切り捨てられていることに注意してください
視覚的な混乱を減らすための小数点の後の数字。
- 同様に、他の数字には切り捨てられた値も含まれている場合があります

- 慣習により、非正規化された注意の重量は**「注意スコア」と呼ばれますが、合計1に合計されている正規化された注意スコアは**「注意重み」と呼ばれます**

- 下のコードは、上の図を段階的に歩きます<br>
-  **ステップ1：**非正規化された注意スコア$ \ omega $を計算します
-2番目の入力トークンをクエリとして使用するとします。つまり、$ q^{（2）} = x^{（2）} $、DOT製品を介して非正規化された注意スコアを計算します。
-  $ \ omega_ {21} = x^{（1）} q^{（2）\ top} $
-  $ \ omega_ {22} = x^{（2）} q^{（2）\ top} $
-  $ \ omega_ {23} = x^{（3）} q^{（2）\ top} $
-  ...
-  $ \ omega_ {2t} = x^{（t）} q^{（2）\ top} $
- 上記では、$ \ omega $はギリシャ文字「オメガ」であり、非正規化された注意スコアを象徴するために使用されます
-  $ \ omega_ {21} $の添え字 "21"は、入力シーケンス要素2が入力シーケンス要素1に対するクエリとして使用されたことを意味します。

- 第3章で説明されているように、3次元ベクターに既に埋め込まれている次の入力文があるとします（ここには、イラストの目的で非常に小さな埋め込みディメンションを使用して、ラインブレークなしでページに適合するように）：

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- （この本では、トレーニングの例が行として列として行われ、上記のテンソルの場合、各行は単語を表し、各列は埋め込み寸法を表す、トレーニングの例が行と特徴値として表される共通の機械学習と深い学習慣習に従います））

- このセクションの主な目的は、コンテキストVector $ z^{（2）} $がどのようにどのように行われるかを示すことです。
クエリとして、2番目の入力シーケンス$ x^{（2）} $を使用して計算されます

- 図は、このプロセスの最初のステップを示しています。
ドット製品操作を通じて他のすべての入力要素

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- コンテキストベクトル$ z^{（2）} $を計算する例として、入力シーケンス要素2、$ x^{（2）} $を使用します。このセクションの後半では、これを一般化して、すべてのコンテキストベクトルを計算します。
- 最初のステップは、クエリ$ x^{（2）} $と他のすべての入力トークンの間でDOT製品を計算することにより、非正規化された注意スコアを計算することです。

In [ ]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- サイドノート：ドット製品は、基本的に2つのベクトル要素を増やし、結果の製品を合計するための速記です。

In [ ]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


-  **ステップ2：**非正規化された注意スコア（ "Omegas"、$ \ omega $）を正規化して、それらが合計1まで合計するように
- 非正規化された注意スコアを正規化して最大1（コンベンション、解釈に役立ち、トレーニングの安定性に重要）を正規化する簡単な方法を次に示します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [ ]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- ただし、実際には、正規化にSoftMax関数を使用して、極端な値の取り扱いに優れており、トレーニング中により望ましい勾配特性を持っていることが一般的で推奨されます。
- スケーリング用のSoftMax関数の素朴な実装です。これにより、ベクトル要素は最大1までの要素を正規化します。

In [ ]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上記の素朴な実装は、オーバーフローとアンダーフローの問題による大小の入力値の数値不安定性の問題に苦しむ可能性があります
- したがって、実際には、代わりにSoftMaxのPytorch実装を使用することをお勧めします。これは、パフォーマンスのために高度に最適化されています。

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


-  **ステップ3 **：埋め込み入力トークンを乗算することにより、コンテキストベクトル$ z^{（2）} $を計算します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [ ]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2すべての入力トークンの注意ウェイトを計算します

####すべての入力シーケンストークンに一般化します：

- 上記では、入力2の注意力とコンテキストベクトルを計算しました（下の図の強調表示された行に示されています）
- 次に、すべての注意の重みとコンテキストベクターを計算するためにこの計算を一般化しています

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">
- （この図の数字は2つに切り捨てられていることに注意してください
視覚的な混乱を減らすための小数点の後の数字。各行の値は、1.0または100％になるはずです。同様に、他の図の数字は切り捨てられます）

- 自己関節では、プロセスは注意スコアの計算から始まり、その後、合計1の注意重量を導き出すように正規化されます
- これらの注意の重みは、入力の加重合計を介してコンテキストベクターを生成するために利用されます

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- 前の**ステップ1 **をすべてのペアワイズ要素に適用して、非正規化された注意スコアマトリックスを計算します。

In [ ]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- マトリックスの乗算により、上記と同じをより効率的に達成できます。

In [ ]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


-  **ステップ2 **と同様に、以前は、各行の値が1に合計されるように各行を正規化します。

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 各行の値が実際に1に合計されていることを迅速に検証します：

In [ ]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 前の**ステップ3 **を適用して、すべてのコンテキストベクトルを計算します。

In [ ]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 正気のチェックとして、以前に計算されたコンテキストベクトル$ z^{（2）} = [0.4419、0.6515、0.5683] $は、上の2行目にあります。

In [ ]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4トレーニング可能なウェイトで自己触たちの実装

- このセクションで開発された自己関節メカニズムが、この本と章の全体的な物語と構造にどのように統合されているかを示す概念的なフレームワーク

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1注意の重みを段階的に計算します

- このセクションでは、元のトランスアーキテクチャ、GPTモデル、およびその他の人気のあるLLMで使用される自己触媒メカニズムを実装しています
- この自己関節メカニズムは「スケーリングされたドット製品の注意」とも呼ばれます
- 全体的なアイデアは以前に似ています。
- 特定の入力要素に固有の入力ベクトル上の加重合計としてコンテキストベクトルを計算したい
- 上記では、注意の重みが必要です
- ご覧のとおり、以前に紹介した基本的な注意メカニズムと比較して、わずかな違いしかありません。
- 最も顕著な違いは、モデルトレーニング中に更新される重量行列の導入です
- これらのトレーニング可能な重量行列は重要であるため、モデル（具体的には、モデル内の注意モジュール）が「良い」コンテキストベクターを生成することを学ぶことができます

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 自己触媒メカニズムを段階的に実装すると、3つのトレーニングウェイトマトリックス$ w_q $、$ w_k $、および$ w_v $を導入することから始めます。
- これらの3つのマトリックスは、埋め込み入力トークン、$ x^{（i）} $、マトリックス乗算を介してクエリ、キー、および値ベクトルに投影するために使用されます。

- クエリベクトル：$ q^{（i）} = w_q \、x^{（i）} $
- キーベクトル：$ k^{（i）} = w_k \、x^{（i）} $
- 値ベクトル：$ v^{（i）} = w_v \、x^{（i）} $

- 入力$ x $とクエリベクトル$ q $の埋め込み寸法は、モデルの設計と特定の実装に応じて、同じまたは異なる場合があります
-GPTモデルでは、入力と出力の寸法は通常同じですが、イラストの目的のために、計算をよりよく追跡するために、ここで異なる入力と出力の寸法を選択します。

In [ ]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

- 以下では、3つの重みマトリックスを初期化します。イラストの目的で出力を減らすために `requires_grad = false`を設定していることに注意してください。しかし、モデルトレーニングにウェイトマトリックスを使用する場合、モデルトレーニング中にこれらのマトリックスを更新するために` requires_grad = true`を設定します

In [ ]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 次に、クエリ、キー、および値ベクトルを計算します。

In [ ]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- 以下に示すように、3Dから6つの入力トークンを2D埋め込みスペースに予測しました。

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 次のステップでは、**ステップ2 **で、クエリと各キーベクトルの間でDOT製品を計算することにより、非正規化された注意スコアを計算します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [ ]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


-  6つの入力があるため、指定されたクエリベクトルに6つの注意スコアがあります。

In [ ]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- 次に、**ステップ3 **で、以前に使用したSoftMax関数を使用して、注意の重み（合計1までの正規化された注意スコア）を計算します
- 以前の違いは、埋め込み寸法の平方根で除算することにより、注意スコアをスケーリングすることです。

In [ ]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

-in **ステップ4 **で、入力クエリベクトル2のコンテキストベクトルを計算します。

In [ ]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2コンパクトな自己競争クラスの実装

- それをすべてまとめると、次のように自己触媒メカニズムを実装できます。

In [ ]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

-Pytorchの線形層を使用して上記の実装を合理化できます。これは、バイアスユニットを無効にするとマトリックスの乗算に相当します。
- マニュアルで `nn.linear`を使用するもう1つの大きな利点` nn.parameter（torch.rand（...） `アプローチは、` nn.linear`が優先重量初期化スキームを持っていることであり、より安定したモデルトレーニングにつながることです

In [ ]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


-  `selfattention_v1`および` selfattention_v2`は、重量行列に異なる初期重みを使用するため、異なる出力を与えることに注意してください

## 3.5将来の単語を因果的な注意を払って隠す

- 因果関係において、対角線の上の注意の重みがマスクされており、特定の入力について、LLMが注意重量でコンテキストベクトルを計算しながら将来のトークンを利用できないことを保証します

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1因果注意マスクの適用

- このセクションでは、以前の自己関節メカニズムを因果的自己触たちメカニズムに変換しています
- 因果的自己触媒により、シーケンス内の特定の位置に対するモデルの予測が、将来の位置ではなく、以前の位置での既知の出力にのみ依存することを保証します
- 簡単に言えば、これにより、次の単語の予測がそれぞれ前の単語のみに依存することが保証されます
- これを達成するために、与えられたトークンごとに、将来のトークン（入力テキストの現在のトークンの後に来るトークン）をマスクします。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- 因果的な自己告発を説明し、実装するには、前のセクションの注意スコアと重みで作業しましょう。

In [ ]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 将来の注意の重みをマスクする最も簡単な方法は、メインの対角線（対角線自体を含む）の下に設定されたメインの対角線設定の上に設定された要素の下にある要素を使用して、PytorchのTRIL関数を介してマスクを作成することです。

In [ ]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 次に、このマスクで注意の重みを掛けて、対角線上の注意スコアをゼロにすることができます。

In [ ]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- ただし、上記のようにソフトマックスの後にマスクが適用された場合、softmaxによって作成された確率分布が破壊されます
-SoftMaxは、すべての出力値の合計が1に保証されます
- ソフトマックスの後のマスキングでは、出力を再び合計するように再び正規化する必要があります。これにより、プロセスが複雑になり、意図しない効果が発生する可能性があります

- 行が1に合計されることを確認するには、次のように注意の重みを正規化できます。

In [ ]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 今すぐ因果注意メカニズムのコーディングを技術的に完了していますが、上記と同じようにより効率的なアプローチを簡単に見てみましょう
- したがって、結果を斜めにゼロをゼロにして結果を繰り返す代わりに、ソフトマックス関数を入力する前に、負の無限で対角線上の非正規化された注意スコアをマスクすることができます。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 以下に示すように、各行の注意の重みは再び正しく1に合計されます。

In [ ]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2ドロップアウトで追加の注意ウェイトをマスキングします

- さらに、トレーニング中の過剰適合を減らすためにドロップアウトも適用します
- ドロップアウトはいくつかの場所で適用できます。
- たとえば、注意の重みを計算した後。
- または注意の重量を値ベクトルで乗算した後
- ここでは、注意の重みを計算した後、ドロップアウトマスクを適用します。

- さらに、この特定の例では、50％のドロップアウト率を使用します。これは、注意力の半分をランダムにマスクアウトすることを意味します。（後でGPTモデルをトレーニングすると、0.1や0.2などの低ドロップアウト率を使用します。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

-  0.5（50％）のドロップアウト率を適用すると、非ドロップ値が1/0.5 = 2の係数によってスケーリングされます。
- スケーリングはF1 /（1- `Dropout_rate`）によって計算されます

In [ ]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [ ]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


- Note that the resulting dropout outputs may look different depending on your operating system; you can read more about this inconsistency [here on the PyTorch issue tracker](https://github.com/pytorch/pytorch/issues/121595)

### 3.5.3コンパクトな因果関係クラスの実装

- 今、私たちは因果関係やドロップアウトマスクを含む、自己attentionの実用的な実装を実装する準備ができています
- もう1つは、「Caselattention」クラスが第2章で実装したデータローダーによって生成されたバッチ出力をサポートするように、複数の入力で構成されるバッチを処理するコードを実装することです。
- 簡単にするために、このようなバッチ入力をシミュレートするために、入力テキストの例を複製します。

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [ ]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs
        # do not exceed `context_length` before reaching this forward method.
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- ドロップアウトは、トレーニング中にのみ適用され、推論中ではないことに注意してください

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6マルチヘッドの注意へのシングルヘッドの注意を拡張します

### 3.6.1複数のシングルヘッドの注意レイヤーを積み重ねます

- 以下は、以前に実装された自己関節の要約です（簡単にするために示されていない因果関係とドロップアウトマスク）

- これはシングルヘッドの注意とも呼ばれます：<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">
- 複数のシングルヘッド注意モジュールを積み重ねて、マルチヘッド注意モジュールを取得するだけです。<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">
- マルチヘッドの注意の背後にある主なアイデアは、異なる学習された線形投影で注意メカニズムを複数回（並行して）実行することです。これにより、モデルは異なる位置の異なる表現サブスペースの情報に共同で参加できます。

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 上記の実装では、埋め込みディメンションは4です。これは、キー、クエリ、値ベクトルの埋め込みディメンション、およびコンテキストベクトルの埋め込みディメンションとして「D_out = 2」です。また、2つの注意ヘッドがあるため、出力埋め込みディメンション2*2 = 4があります。

### 3.6.2重量スプリットでマルチヘッドの注意を実装します

- 上記は、マルチヘッドの注意の直感的で完全に機能的な実装ですが（以前からの単一ヘッドの注意「因果関係」実装を包む）、「MultiHeadttention」と呼ばれるスタンドアロンクラスを書くことができます。

- このスタンドアロンの「MultiHeadattention」クラスの単一の注意ヘッドを連結していません
- 代わりに、単一のw_query、w_key、およびw_value重量行列を作成し、各注意ヘッドの個々のマトリックスにそれらを分割します。

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`,
        # this will result in errors in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs
        # do not exceed `context_length` before reaching this forwar

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 上記は本質的に「multiheadtentionwrapper」の書き換えバージョンであることに注意してください。
- 結果の出力は、ランダムな重みの初期化が異なるため、少し異なって見えますが、両方とも今後の章で実装するGPTクラスで使用できる完全に機能的な実装です
- さらに、上記の「MultiHeadattention」クラスに線形投影層（ `self.out_proj`）を追加したことに注意してください。これは、寸法を変えない単なる線形変換です。LLM実装でこのような投影層を使用することは標準的な慣習ですが、厳密には必要ありません（最近の研究では、モデリングパフォーマンスに影響を与えることなく削除できることが示されています。この章の最後にある読書セクションを参照してください）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- Note that if you are interested in a compact and efficient implementation of the above, you can also consider the [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) class in PyTorch

- 上記の実装は一見して少し複雑に見えるかもしれないので、 `attn_scores = queries @ keys.Transpose（2、3）`：を実行するときに何が起こるかを見てみましょう。

In [ ]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- この場合、Pytorchのマトリックス乗算実装は、4次元入力テンソルを処理して、マトリックスの乗算が2つの最後の寸法（num_tokens、head_dim）の間で実行され、個々のヘッドのために繰り返されるようになります。

- たとえば、以下は、各ヘッドのマトリックス乗算を個別に計算するためのよりコンパクトな方法になります。

In [ ]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


＃概要とテイクアウト

- See the [./multihead-attention.ipynb](./multihead-attention.ipynb) code notebook, which is a concise version of the data loader (chapter 2) plus the multi-head attention class that we implemented in this chapter and will need for training the GPT model in upcoming chapters
- You can find the exercise solutions in [./exercise-solutions.ipynb](./exercise-solutions.ipynb)